# ASSUME CAS NOT REUSED, STAGE OF CELL CYCLE ALLOWS CRISPR-CAS

# p = P(MUTATE (OFF)-TARGET) = mCpPAM x P(MUTATE | PAM)

# FIND m

In [1]:
#m = # sgRNA-Cas molecules
#Hsu et al. ->
m=180660

# C IS PARAMETER TO FIT

In [2]:
#C = (1-b^N)/(1-b)
#b = P(1 trial results in no mutation) = constant for given genome
#N = max trials possible by CRISPR-Cas in time interval studied = constant

# FIND pPAM

In [3]:
#pPAM = P(interrogate particular PAM in 1 trial)
#1 trial: sgRNA interrogates 1 PAM -> mutate or no mutate

#Hsu et al. -> human cells

# FASTA SEQUENCE IS + STRAND, 5'-3'

# VALID TARGET+PAM IS ANY 5'-TARGET-PAM-3'

In [4]:
def complement35(DNA53): #given a 5'-3' DNA strand as string, return the 3'-5' complement as string
    k=0
    DNA35 = ""
    while k<len(DNA53):
        if DNA53[k]=='A':
            DNA35 += 'T'
        elif DNA53[k]=='T': #must use elif, not else!
            DNA35 += 'A'
        elif DNA53[k]=='C':
            DNA35 += 'G'
        else:
            DNA35 += 'C'
        k+=1
    return DNA35

In [5]:
def reverse(DNA): #given a string, return it in reverse order
    k=len(DNA)-1
    rev = ""
    while k>=0:
        rev+=DNA[k]
        k-=1
    return rev

# OCCURENCE OF PAMS IN HUMAN REFERENCE GENOME GRCh38.p4

# ASSUME SINGLE-COPIED CHROMOSOMES: 1-22, X, av(X,Y)

In [6]:
from Bio import SeqIO
import os
os.chdir("/Users/lickivy/Research/LladserYin/Python/GRCh38.p4")

In [7]:
#PAMs are NRG: NAG, NGG
#takes 3 min
NAG=0
NGG=0
k=1
while k<23:
    record=SeqIO.read("humanchromosome"+str(k)+".fasta","fasta")
    seq=record.seq
    for N in "ATCG": #RG must be preceded by some N
        PAM=N+"AG" #fasta for 5'NAG3'+
        PAM_2=reverse(complement35(PAM)) #fasta for 5'NAG3'-
        NAG+=seq.count(PAM)+seq.count(PAM_2)
        PAM=N+"GG" #fasta for 5'NGG3'+
        PAM_2=reverse(complement35(PAM)) #fasta for 5'NGG3'-
        NGG+=seq.count(PAM)+seq.count(PAM_2)
    k+=1
print("# NAGs in autosomes =",NAG)
print("# NGGs in autosomes =",NGG)

record=SeqIO.read("humanchromosomeX.fasta","fasta")
seq=record.seq
NAGinX=0
NGGinX=0
for N in "ATCG": #RG must be preceded by some N
        PAM=N+"AG" #fasta for 5'NAG3'+
        PAM_2=reverse(complement35(PAM)) #fasta for 5'NAG3'-
        NAGinX+=seq.count(PAM)+seq.count(PAM_2)
        PAM=N+"GG" #fasta for 5'NGG3'+
        PAM_2=reverse(complement35(PAM)) #fasta for 5'NGG3'-
        NGGinX+=seq.count(PAM)+seq.count(PAM_2)
print("# NAGs in X =",NAGinX)
print("# NGGs in X =",NGGinX)
        
record=SeqIO.read("humanchromosomeY.fasta","fasta")
seq=record.seq
NAGinY=0
NGGinY=0
for N in "ATCG": #RG must be preceded by some N
        PAM=N+"AG" #fasta for 5'NAG3'+
        PAM_2=reverse(complement35(PAM)) #fasta for 5'NAG3'-
        NAGinY+=seq.count(PAM)+seq.count(PAM_2)
        PAM=N+"GG" #fasta for 5'NGG3'+
        PAM_2=reverse(complement35(PAM)) #fasta for 5'NGG3'-
        NGGinY+=seq.count(PAM)+seq.count(PAM_2)
print("# NAGs in Y =",NAGinY)
print("# NGGs in Y =",NGGinY)
        
NAG=2*NAG+NAGinX+(NAGinX+NAGinY)/2
NGG=2*NGG+NGGinX+(NGGinX+NGGinY)/2

print("# NAG =",NAG)
print("# NGG =",NGG)

# NAGs in autosomes = 380034908
# NGGs in autosomes = 269822755
# NAGs in X = 20843230
# NGGs in X = 14166478
# NAGs in Y = 3535774
# NGGs in Y = 2474436
# NAG = 793102548.0
# NGG = 562132445.0


In [8]:
os.chdir("/Users/lickivy/Research/LladserYin/Python")

# pPAM

In [9]:
# pPAM = pNAG, pNGG
# (number of NAGs) x pNAG + (number of NGGs) x pNGG = 1
# pNAG=pNGG/5
pNAG=1/(NAG+5*NGG)
pNGG=5*pNAG
print("pNAG =",pNAG)
print("pNGG =",pNGG)
NAG*pNAG+NGG*pNGG # =1 works

pNAG = 2.7748758950422207e-10
pNGG = 1.3874379475211104e-09


1.0

# SIMPLE MODEL (see simple model notes): P(MUTATE | PAM)

# DOUBLE CLICK TO MAKE COHERENT:

target:             5' ...PAM 3'
target complement:  3' ...    5'
gRNA:               5' ...    3'

# COMPUTE MATCHES (1'S AND 0'S)

In [10]:
def matchesPAM(target53,gRNA53): 
#given 5'-3' target + PAM and 5'-3' gRNA + PAM as strings,
#check same length, 
#and return 5'-3' sequence of 1's (bp matches) and 0's (bp mismatches) + target PAM (AGTC's)
    N=len(target53)
    if N != len(gRNA53):
        print("lengths don't match")
    else:
        
        i=0
        m="" #initialize list
        while i<N-3:
            if gRNA53[i] == "U":
                if target53[i]== "T":
                    m += "1"
                else:
                    m += "0"
            else:
                if target53[i]==gRNA53[i]:
                    m += "1"
                else:
                    m += "0"
            i+=1
            
        PAM=""
        while i<N:
            PAM += target53[i]
            i+=1
            
        return m+PAM

In [11]:
matchesPAM("A","AU")

lengths don't match


In [12]:
matchesPAM("ACCGTT","ACCGUU")

'111GTT'

In [13]:
matchesPAM("TTTACCGAT","UGCACCGUU")

'100111GAT'

# DOUBLE CLICK TO MAKE COHERENT:

target:                   5' ...         PAM  3'
target complement:        3' ...              5'
position n:               5' 19,18,...0       3'
matchesPAM:               5' 0,0,1,1...       3'    =    5' x_19, x_18, ..., x_1, x_0 3'
consecutive mismatches k: 5' 2,1,1,1...       3'
x_n = 1 is match at position n, x_n = 0 is mismatch at position n

# LOG PROB OF NT BINDING (LN(P_N))

In [14]:
def ltol(s,w,delta,n,k):
#given parameters s, w, delta,
#and position n (count up from 0 going 3'-5' along target after PAM),
#and number of consecutive mismatches k (>= 1),
#defines ln[P(tolerate mismatch at position n)] = ln[P(bind nt | x_n = 0)]
    return -s*(k-1)-w*delta**n

# LOG PROB OF MUTATION (L)

In [15]:
def l(matchesPAM,s,w,delta):
#NOTE s in (0,infty), w in (0,infty), delta in (0,1)
#given 5'-3' (along target DNA) string, matchesPAM, of mismatches (0) and matches (1) followed by PAM (AGTC's),
#and function ltol = ln[P(tolerate mismatch)],
#and assuming matches always bind,
#and assuming PAM not equal to NRG (N={A,G,T,C}, R={A,G}, Y={T,C}) never binds,
#check PAM is NRG (5'-3' along target DNA),
#and return ln[P(mutate|PAM)]
    N=len(matchesPAM)
    if matchesPAM[N-1] != "G" or (matchesPAM[N-2] in ("A","G")) == False:
        return "INVALID PAM"
    else:
        l=0 #ln[P(mutate|PAM)]
        n=0 #position
        k=1 #consecutive MM's, k>=1
        if matchesPAM[N-1-3]=="0":
            l=l+ltol(s,w,delta,n,k)
            n+=1
        else:
            n+=1
        while n<N-3:
            if matchesPAM[N-1-3-n] == "0": #MUST include ""
                if matchesPAM[N-1-3-n+1] == "0": 
                    k+=1
                else:
                    k=1
                l=l+ltol(s,w,delta,n,k)
            else:
                k=1
            n+=1
    return l

In [16]:
print(l("001000110TTG",0,1,1)) #PAM works
print("INVALID PAM")

INVALID PAM
INVALID PAM


In [17]:
print(l("0010001100TGG",1,0,.5)) #s,k work
print(-1-1-2-1)

-5.0
-5


In [18]:
print(l("001000110TGG",0,.5,2)) #w,delta work
print(-.5*(2**0+2**3+2**4+2**5+2**7+2**8))

-220.5
-220.5


# PROBABILITY OF BINDING FOR EXAMPLE IN WRITEUP, NO PROBLEM WITH FLOATING PT PRECISION

In [19]:
#s=1,w=1,delta=.5
print(l("10011100011TGG",1,1,.5))
print(-(1*(1-1)+1*.5**2+2-1+.5**3+3-1+.5**4+1-1+.5**8+2-1+.5**9))
print(-(3-1+.5**4+
        1-1+.5**8+2-1+.5**9+1*(1-1)+1*.5**2+2-1+.5**3))

-4.443359375
-4.443359375
-4.443359375


In [20]:
import numpy

# COMPUTE MODEL VALUES OF L

In [21]:
def xt(matchesPAM):
    #can handle different sequence lengths
    #given above matchesPAM_t, return x_t = [u_t,m_{t,0},m_{t,1},...,m_{t,len_t-1},0...] = tth row of matrix x
    #u_t = sum_0^{target sequence length - 1} (k_{t,n}-1)
    #m_{t,n} = 1 if nth nt mismatch, 0 if match
    N=len(matchesPAM) #20nt target + 3nt PAM
    x=numpy.zeros(N-3+1)
    if matchesPAM[N-1] != "G" or (matchesPAM[N-2] in ("A","G")) == False:
        return "INVALID PAM"
    else:
        x[0]=l(matchesPAM,-1,0,0)
        n=0
        while n<N-3:
            if matchesPAM[N-3-1-n]=="0":
                x[1+n]=1
            n+=1
        return x

In [22]:
print(xt("100AGG")) #works
print(numpy.array([1,1,1,0]))
print(xt("10AGG"))
print(numpy.array([0,1,0]))

[ 1.  1.  1.  0.]
[1 1 1 0]
[ 0.  1.  0.]
[0 1 0]


In [23]:
def x(matchesPAMs):
    #given array matchesPAMs listing strings matchesPAM_t = [...,matchesPAM_t,...]
    #return matrix x of arrays x_t as rows
    M=len(matchesPAMs)
    N=len(matchesPAMs[0])
    n=0
    while n<M:
        if len(matchesPAMs[n]) > N:
            N=len(matchesPAMs[n])
        n+=1
    x=numpy.zeros([M,N-3+1])
    n=0
    while n<M:
        y=xt(matchesPAMs[n])
        if y=="INVALID PAM":
            return "INVALID PAM"
        else:
            x[n,:]=numpy.append(y,numpy.zeros(N-3+1-len(y)))
        n+=1
    return x

In [24]:
x(["111GAA","110AGG"]) #PAM works

'INVALID PAM'

In [25]:
print(xt("100AGG")) #rest works
print(xt("10AGG"))
print(xt("001AGG"))
print(xt("101AGG"))
matchesPAMs=["100AGG","10AGG","001AGG","101AGG"]
X=x(matchesPAMs)
X

[ 1.  1.  1.  0.]
[ 0.  1.  0.]
[ 1.  0.  1.  1.]
[ 0.  0.  1.  0.]


array([[ 1.,  1.,  1.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 1.,  0.,  1.,  1.],
       [ 0.,  0.,  1.,  0.]])

In [26]:
def ymodel(theta,matchesPAMs):
    #given list theta = [s,w,delta], matchesPAMs
    #return numpy.array ymodel of model y-values = [l(x_1),...,l(x_t),...,l(x_N)]
    s,w,delta=theta
    X=x(matchesPAMs)
    y=-s*X[:,0]
    n=0
    xwidth=len(X[0,:])
    while n < xwidth-1:
        y += -w*X[:,n+1]*delta**n
        n+=1 
    return y

In [27]:
theta=[2,2,1]
print(X)
print(numpy.array([-2*1-2*(1+1),-2*0-2*(1),-2*1-2*(1+1)]))
Ymodel=ymodel(theta,matchesPAMs)
print(Ymodel) #works

[[ 1.  1.  1.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  1.  1.]
 [ 0.  0.  1.  0.]]
[-6 -2 -6]
[-6. -2. -6. -2.]


# COMPUTE PSEUDODATA VALUES OF L

In [28]:
def ypseudo(theta,matchesPAMs,sigma):
    #given list theta = [s,w,delta], matchesPAMs, sigma = sd of error
    #return numpy.array ypseudo of pseudodata y-values = [l_1,...,l_t,...,l_N]
    y1=ymodel(theta,matchesPAMs)
    y2=y1+numpy.random.normal(0,sigma,len(y1)) #normal(mean, s.d, length)
    return y2

In [29]:
Ypseudo=ypseudo(theta,matchesPAMs,.5)
print(Ypseudo)

[-6.10157312 -1.69777827 -5.16952351 -2.39632954]


# p = P(MUTATE (OFF)-TARGET) = mCpPAM x P(MUTATE | PAM)

# COMPUTE MODEL VALUES OF LN(p)

In [30]:
def lnpmodel(C,theta,matchesPAMs):
    #given C, list theta = [s,w,delta], matchesPAMs
    #return numpy.array lnpmodel of model ln(p)-values = [lnp(C,x_1),...,lnp(C,x_t),...,lnp(C,x_N)]
    return numpy.log(C*m)+ymodel(theta,matchesPAMs)

In [31]:
C=2
Lnpmodel=lnpmodel(C,theta,matchesPAMs)
print(Lnpmodel)
print(numpy.log(C*m)+Ymodel) #works

[  6.79751927  10.79751927   6.79751927  10.79751927]
[  6.79751927  10.79751927   6.79751927  10.79751927]


# COMPUTE PSEUDODATA VALUES OF LN(p)

In [32]:
def lnppseudo(C,theta,matchesPAMs,sigma):
    #given C, list theta = [s,w,delta], matchesPAMs, sigma = sd of error
    #return numpy.array lnppseudo of pseudodata ln(p)-values = [lnp_1,...,lnp_t,...,lnp_N]
    y1=lnpmodel(C,theta,matchesPAMs)
    y2=y1+numpy.random.normal(0,sigma,len(y1)) #normal(mean, s.d, length)
    return y2

In [33]:
Lnppseudo=lnppseudo(C,theta,matchesPAMs,.5)
print(Lnppseudo)

[  6.69734245  11.3710995    6.11655909   9.6856048 ]


# LEAST SQUARES FIT OF MODEL TO DATA LN(p) VALUES

In [34]:
def residuals(Cthetahat,y,x):
    #given:
    #list Cthetahat of model param estimates = [C,s,w,delta],
    #numpy.array y of data ln(p)-values = [...,lnp_t,...],
    #matrix numpy.array x of data x-values = [...,x_t,...]
    #return:
    #numpy.array of residuals = data - model ln(p)-values = [...,lnp_t-lnp(x_t),...]
    C,s,w,delta=Cthetahat
    xwidth=len(x[0,:])
    res=y-numpy.log(C*m)+x[:,0]*s
    n=0
    while n < xwidth-1:
        res += w*x[:,n+1]*delta**n
        n+=1
    return res

In [35]:
print(residuals([C,2,2,1],Lnpmodel,X)) #works
print(numpy.array([0,0,0]))
print(residuals([C,2,2,1],Lnppseudo,X))
print(Lnppseudo-Lnpmodel)

[ 0.  0.  0.  0.]
[0 0 0]
[-0.10017682  0.57358023 -0.68096018 -1.11191447]
[-0.10017682  0.57358023 -0.68096018 -1.11191447]


In [36]:
from scipy.optimize import leastsq

see http://docs.scipy.org/doc/scipy-0.14.0/reference/tutorial/optimize.html > Least-square fitting

In [37]:
def LS(Ctheta0,y,x):
    #given data y = [...,lnp_t,...], data x = [...,x_t,...], initial guess Ctheta0 = [C0,s0,w0,delta0], and residuals function
    #return estimated params = numpy array [C, s, w, delta]    
    #NOTE: need at least as many data values as parameters
    Cthetahat=leastsq(residuals,Ctheta0,args=(y,x))[0]
    #print("[C,s,w,delta] = ")
    return(Cthetahat)

# IF RUNTIME ERROR ENCOUNTERED IN LOG, JUST RERUN LS FIT

In [38]:
print(LS([-10,-10,-10,-10],Lnpmodel,X)) #fit is not necessarily unique, must try different theta0 to make s in (0,infty), w in (0,infty), delta in (0,1)  
print(LS([2,2,2,1],Lnpmodel,X)) #works
print(LS([2,2,2,2],Lnpmodel,X)) #ok fit...
print(LS([5,5,5,5],Lnpmodel,X)) #bad fit
print([2,2,2,1])

[-10. -10. -10. -10.]
[ 2.  2.  2.  1.]
[ 0.95567596  2.73848355  1.26151645  1.        ]
[  2.70670566e-01   4.00000000e+00  -2.92557971e-75   3.09180036e+00]
[2, 2, 2, 1]


-c:10: RuntimeWarning: invalid value encountered in log


# LEAST SQUARES FIT OF MODEL TO PSEUDODATA LN(p) VALUES: EXAMPLE

In [39]:
C=2
theta=(2,.333,.5)
Ctheta=[2,2,.333,.5]
Ctheta0=[1,1,1,10] #initial guess of theta = (s,w,delta)
sigma=0.1 #sd of error of pseudodata
matchesPAMs=[]
for a in "01":
    for b in "01":
        for c in "01":
            for d in "01":
                for e in "01":
                    if (a+b+c) != "010" and d != "0" or e==1: #removing one of the graphs manually
                        matchesPAMs.append(a+b+c+d+e+"AGG")
print("Ctheta = ",Ctheta)
print("Ctheta0 = ",Ctheta0)
print("matchesPAMs = ",matchesPAMs)
Lnpmodel=lnpmodel(C,theta,matchesPAMs)
X=x(matchesPAMs)
Lnppseudo=lnppseudo(C,theta,matchesPAMs,sigma)
print("Lnpmodel = ",Lnpmodel)
print("Lnppseudo = ",Lnppseudo)
print("X = ",x(matchesPAMs))

Ctheta =  [2, 2, 0.333, 0.5]
Ctheta0 =  [1, 1, 1, 10]
matchesPAMs =  ['00010AGG', '00011AGG', '00110AGG', '00111AGG', '01110AGG', '01111AGG', '10010AGG', '10011AGG', '10110AGG', '10111AGG', '11010AGG', '11011AGG', '11110AGG', '11111AGG']
Lnpmodel =  [  6.31883177   6.65183177  10.40208177  10.73508177  12.44370677
  12.77670677  10.33964427  10.67264427  12.42289427  12.75589427
  12.38126927  12.71426927  12.46451927  12.79751927]
Lnppseudo =  [  6.36060475   6.52496125  10.44289765  10.74311179  12.35391391
  12.85169686  10.36280885  10.61483672  12.50408809  12.76133973
  12.33439395  12.83465495  12.56643662  12.85705678]
X =  [[ 3.  1.  0.  1.  1.  1.]
 [ 3.  0.  0.  1.  1.  1.]
 [ 1.  1.  0.  0.  1.  1.]
 [ 1.  0.  0.  0.  1.  1.]
 [ 0.  1.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 1.  1.  0.  1.  1.  0.]
 [ 1.  0.  0.  1.  1.  0.]
 [ 0.  1.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.  0.]
 [ 0.  1.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.]
 [ 0.  0.

In [40]:
LS(Ctheta0,Lnppseudo,X) #good fit

array([ 2.13395289,  1.99693576,  0.32216288,  0.63260171])

In [41]:
LS(Ctheta0,Lnpmodel,X)

array([ 2.   ,  2.   ,  0.333,  0.5  ])

# LEAST SQUARES MODEL FIT TO HSU ET AL. FIG2 DATA: DIFFERENT % DATA USED

# CLEAR FIT# COLS IN EXCEL BEFORE RUNNING

# IF ERROR, JUST RERUN

# RANDOM SAMPLES -> DIFFERENT RESULTS EACH RUN:

In [42]:
from openpyxl import Workbook
from openpyxl import load_workbook
import math
from numpy import random as ran

In [44]:
#takes 5 min

#simple model:
#use increments of 10% of data to fit model parameters
#predict unused data p estimates
#calculate SSE (prediction vs. data)

wb=load_workbook('SuppTable5fit.xlsx')
ws=wb.active
ws['CC5'].value="Sample #'s used = fitted" #col 81
ws['CD5'].value="Sample #'s unused = predicted"
ws['CE5'].value="C"
ws['CF5'].value="s"
ws['CG5'].value="w"
ws['CH5'].value="delta"
ws['CI5'].value="SSE (pmodel vs. pdata)"
ws['CJ5'].value="RMSE (pmodel vs. pdata)"
ws['CM5'].value="lnpdata"
ws['CN5'].value="Python lnpdata (python doesn't accept excel formulas)"
ws['CO5'].value="matchesPAM" #col 93

#calculate matchesPAM
tot=ws['CB8'].value
k=0
while k<tot:
    r=6+k
    ws.cell(row=r,column=93).value=matchesPAM(ws.cell(row=r,column=68).value+"GGG",ws.cell(row=r,column=70).value+"GGG")
    k+=1   

SSE=numpy.array([])
RMSe=numpy.array([])
percent=numpy.array([])
    
tenpercent=1 #use tenpercent*10 % of data
while tenpercent<10:
    amount=math.floor(tenpercent*10/100*tot) #use amount # of data
    
    #do T times for each %data
    #pwr analysis -> T=310
    T=310
    k=0
    while k<T:
        #use sample #'s in used:
        used=numpy.array([])
        unused=numpy.array([])
        i=0
        while i<amount:
            num=ran.randint(1,tot+1)
            while num in used: #use 10% of data unique data
                num=ran.randint(1,tot+1)
            used=numpy.append(used,num)
            i+=1
        i=1
        while i<tot+1:
            if i not in used:
                unused=numpy.append(unused,i)
            i+=1
        fitnum=T*(tenpercent-1)+k #fit number
        r=6+fitnum #row number
        ws.cell(row=r,column=81).value=numpy.array_str(used)
        ws.cell(row=r,column=82).value=numpy.array_str(unused)
        
        #fit used data
        #used row = r
        matchesPAMs=numpy.array([])
        Y=numpy.array([])
        N=len(used)
        j=0
        while j<N:
            #matchesPAM row = r2
            r2=int(5+used[j])
            matchesPAMs=numpy.append(matchesPAMs,[ws.cell(row=r2,column=93).value])
            Y=numpy.append(Y,ws.cell(row=r2,column=92).value) #lnpdata
            j+=1
        X=x(matchesPAMs)
        Ctheta0=[2,2,2,2] ####################### initial guess, change to make fits better ########################
        Cthetahat=LS(Ctheta0,Y,X)
        C=Cthetahat[0]
        s=Cthetahat[1]
        w=Cthetahat[2]
        delta=Cthetahat[3]
        ws.cell(row=r,column=83).value=C
        ws.cell(row=r,column=84).value=s
        ws.cell(row=r,column=85).value=w
        ws.cell(row=r,column=86).value=delta
        
        #predict unused data
        matchesPAMs=numpy.array([])
        Pdata=numpy.array([])
        M=len(unused)
        j=0
        while j<M:
            #matchesPAM row = r2
            r2=int(5+unused[j])
            matchesPAMs=numpy.append(matchesPAMs,[ws.cell(row=r2,column=93).value])
            Pdata=numpy.append(Pdata,ws.cell(row=r2,column=90).value) #pdata
            j+=1
        theta=[s,w,delta]
        Pmodel=numpy.exp(lnpmodel(C,theta,matchesPAMs)) #pmodel   
        #pmodel fit col = s
        s=94+fitnum
        ws.cell(row=5,column=s).value="pmodel Fit"+str(fitnum+1)
        j=0
        while j<M:
            #matchesPAM row = r2
            r2=int(5+unused[j])
            ws.cell(row=r2,column=s).value=Pmodel[j]
            j+=1
        
        #compute SSE
        SSEval=sum((Pdata-Pmodel)*(Pdata-Pmodel))
        ws.cell(row=6+fitnum,column=87).value=SSEval
        SSE=numpy.append(SSE,SSEval)
        percent=numpy.append(percent,tenpercent*10)
        
        #compute RMSE
        RMSeval=(SSEval/M)**(1/2)
        ws.cell(row=6+fitnum,column=88).value=RMSeval
        RMSe=numpy.append(RMSe,RMSeval)
        
        k+=1
    tenpercent+=1
wb.save('SuppTable5fit.xlsx')

In [45]:
import matplotlib.pyplot as plt

In [ ]:
#"SSE vs percent.png"
#graph SSE vs. % data used
plt.plot(percent,SSE,'b.')
plt.title("SSE of p \n (Fit parameters to used data, model prediction for unused data)",fontsize=16)
plt.xlabel("% Data Used "+r"($n_i=310$)"+"\n", fontsize=16)
plt.ylabel("SSE", fontsize=16)
plt.show()

In [49]:
avRMSe=numpy.array([])
sdRMSe=numpy.array([])
k=0
while k<9:
    RMSek=RMSe[k*T:k*T+T]
    av=sum(RMSek)/T
    avRMSe=numpy.append(avRMSe,av)
    sd=sum((RMSek-av)*(RMSek-av))/(T-1)
    sdRMSe=numpy.append(sdRMSe,sd)
    k+=1

In [50]:
#"RMSe vs percent.png"
#graph RMS error + error bars vs. % data used
per=numpy.array([10,20,30,40,50,60,70,80,90])
plt.plot(per,avRMSe,'b.')
plt.errorbar(per,avRMSe,yerr=sdRMSe,linestyle="None",elinewidth=6,capsize=5,color="black")
plt.title("RMS Error of p \n(Fit parameters to used data, model prediction for unused data)",fontsize=16)
plt.xlabel("% Data Used "+r"($n_i=310$)"+"\n", fontsize=16)
plt.ylabel(r'$\sqrt{MSE}$', fontsize=16)
plt.show()

# LEAST SQUARES MODEL FIT TO HSU ET AL. FIG2 DATA: DIFFERENT TARGET DATA USED

# DON'T CHANGE:

In [41]:
Ctheta0=[6,6,6,6]

#Target1
matchesPAMs=numpy.array([])
Y=numpy.array([])
i=6
while i<62:
    matchesPAMs=numpy.append(matchesPAMs,ws.cell(row=i,column=92).value)
    Y=numpy.append(Y,ws.cell(row=i,column=91).value)
    i+=1
i=222
while i<275:
    matchesPAMs=numpy.append(matchesPAMs,ws.cell(row=i,column=92).value)
    Y=numpy.append(Y,ws.cell(row=i,column=91).value)
    i+=1
X=x(matchesPAMs)
Cthetahat1=LS(Ctheta0,Y,X)

#Target2
matchesPAMs=numpy.array([])
Y=numpy.array([])
i=62
while i<118:
    matchesPAMs=numpy.append(matchesPAMs,ws.cell(row=i,column=92).value)
    Y=numpy.append(Y,ws.cell(row=i,column=91).value)
    i+=1
i=275
while i<330:
    matchesPAMs=numpy.append(matchesPAMs,ws.cell(row=i,column=92).value)
    Y=numpy.append(Y,ws.cell(row=i,column=91).value)
    i+=1
X=x(matchesPAMs)
Cthetahat2=LS(Ctheta0,Y,X)

#Target3
matchesPAMs=numpy.array([])
Y=numpy.array([])
i=118
while i<171:
    matchesPAMs=numpy.append(matchesPAMs,ws.cell(row=i,column=92).value)
    Y=numpy.append(Y,ws.cell(row=i,column=91).value)
    i+=1
i=330
while i<381:
    matchesPAMs=numpy.append(matchesPAMs,ws.cell(row=i,column=92).value)
    Y=numpy.append(Y,ws.cell(row=i,column=91).value)
    i+=1
X=x(matchesPAMs)
Cthetahat3=LS(Ctheta0,Y,X)

#Target6
matchesPAMs=numpy.array([])
Y=numpy.array([])
i=171
while i<222:
    matchesPAMs=numpy.append(matchesPAMs,ws.cell(row=i,column=92).value)
    Y=numpy.append(Y,ws.cell(row=i,column=91).value)
    i+=1
i=381
while i<435:
    matchesPAMs=numpy.append(matchesPAMs,ws.cell(row=i,column=92).value)
    Y=numpy.append(Y,ws.cell(row=i,column=91).value)
    i+=1
X=x(matchesPAMs)
Cthetahat6=LS(Ctheta0,Y,X)

print("target1 C,s,w,delta =",Cthetahat1)
print("target2 C,s,w,delta =",Cthetahat2)
print("target3 C,s,w,delta =",Cthetahat3)
print("target6 C,s,w,delta =",Cthetahat6)


target1 C,s,w,delta = [  1.26397591e+08   6.00000000e+00   7.27646758e-08   5.80077656e+00]
target2 C,s,w,delta = [  1.18684894e+08   6.00000000e+00   1.20989129e-08   5.78277019e+00]
target3 C,s,w,delta = [  7.49495057e+07   6.00000000e+00   1.60527643e-08   5.78285015e+00]
target6 C,s,w,delta = [  5.30974571e+07   6.00000000e+00   7.37404254e-07   5.78327615e+00]


# NEXT: FIT MARKOV MODEL E

# NEXT: FIT ALL HSU ET AL. DATA